In [ ]:
from IPython.core.display import HTML
import random

def css_styling():
    styles = open("../static/css/custom.css", "r").read()
    return HTML(styles)
css_styling()

### Why Python?

- free, open source
- one platform for data pre-processing, visualization and analysis
- reproducible code
- large number of user-developed packages (eg. nibabel, nilearn)
- easy interaction with state-of-the art neuroimaging software (eg. FSL, ANTS)

### Types of MR Scans

<img src="../static/images/mr_scan_types.png" alt="Drawing" align="middle" width="500px"/>

For this tutorial, we'll be focusing on T1w and resting state fMRI scans.

### Neuroimaging File Formats

|Format Name | File Extension | Origin |
|---|---|---|
| Analyze | .img/.hdr | Analyze Software, Mayo Clinic |
| DICOM | none | ACR/NEMA Consortium |
| NIfTI | .nii or .img/.hdr | Neuroimaging Informatics Technology Initiative |
| MINC | .mnc | Montreal Neurological Institute |
| NRRD | .nrrd | |

<img src="../static/images/dicom_to_nifti.png" alt="Drawing" align="middle" width="300px"/>

From the MRI scanner, images are initially collected in the DICOM format and can be converted to NIfTI using [dcm2niix](https://github.com/rordenlab/dcm2niix).

### Intro to NIfTI

NIfTI is one of the most ubiquitous file formats for storing neuroimaging data. We'll cover a few details to get started working with them. If you're interested in learning more about NIfTI images, we highly recommend [this blog post about the NIfTI format](http://brainder.org/2012/09/23/the-nifti-file-format/).

### Reading NIfTI Images

[NiBabel](http://nipy.org/nibabel/) is a Python package for reading and writing neuroimaging data. To learn more about how NiBabel handles NIfTIs, check out the [Working with NIfTI images](http://nipy.org/nibabel/nifti_images.html) page of the NiBabel documentation.

In [ ]:
import nibabel as nib

First, use the `load()` function to create a NiBabel image object from a NIfTI file. We'll load in a T1w image from the dataset we'll be using for this tutorial.

In [ ]:
t1_img = nib.load('../data/ds000030/sub-10171/anat/sub-10171_T1w.nii.gz')
type(t1_img)

There are three main components of a NIfTI image:

#### 1. [Header](http://nipy.org/nibabel/nibabel_images.html#the-image-header): contains metadata about the image, such as image dimensions, data type, etc.

In [ ]:
t1_hdr = t1_img.header

You can easily access specific metadata from the NiBabel image header object through dictionary keys.

In [ ]:
t1_hdr.keys()

We can access any key in an image through the following methods (there are two ways!):


```python

t1_hdr['key']
t1_hdr.key

```

Both ways work!

<div class=exercise>
    <b>EXERCISE:</b> Extract the value of <code>pixdim</code> from <code>nii_hdr</code>  
</div>

<div class=solution>
    <b>SOLUTION:</b>
</div>

In [ ]:
t1_hdr['pixdim']

#### 2. Data
The data is a multidimensional array representing the image data.

In [ ]:
t1_data = t1_img.get_data()
t1_data

The data is stored in a numpy array.

In [ ]:
type(t1_data)

We can check some basic properties of the array.

<div class=exercise>
    <b>EXERCISE:</b> How many dimesnions does <code>t1_data</code> have? What are is the size of each dimension? What is the data type?
</div>

<div class=solution>
    <b>SOLUTION:</b>
</div>

In [ ]:
t1_data.ndim

In [ ]:
t1_data.shape

The shape of the data always has at least 3 dimensions (X, Y, and Z) and sometimes a 4th, T (time).  
This T1w image has 3 dimensions. The brain was scanned in 176 slices with a resolution of 256 x 256 voxels per slice.

In [ ]:
t1_data.dtype

The data type of an image controls the range of possible intensities. As the number of possible values increases, the amount of space the image takes up in memory also increases.

| Data Type | Range | Number of Values |
|---|---|---|
| uint8 | 0, 255 | 256 |
| uint16 | -128, 127 | 256 |
| uint 16 | 0, 2^16 | 2^16 |
| int16 | -2^15, 2^15 | 2^16 |
| float16 | ~-2^16, ~2^16 | >>2^16 |

#### 3. [Affine](http://nipy.org/nibabel/coordinate_systems.html): tells the position of the image array data in a *reference space*

The affine array tells the position of the image array data in a *reference space*. It translates between data-space and world-space.

In [ ]:
t1_affine = t1_img.affine
t1_affine

<div class=exercise>
    <b>EXERCISE:</b> Explore some of the other methods that can be called on the NIfTI image.
</div>

### Working With Image Data

#### Slicing

n-dimensional images are just stacks of numpy arrays.  Each value in the array is assigned to an x, y or z coordinate.  
<img src="../static/images/numpy_arrays.png" alt="Drawing" align="middle" width="500px"/>

You'll recall our example T1w image is a 3D image with dimensions $176 \times 256 \times 256$.

Slicing does exactly what it seems to imply. Giving our 3D volume, we pull out a 2D slice of our data. Here's an example of slicing from left to right (**sagittal slicing**):

<img src="https://upload.wikimedia.org/wikipedia/commons/5/56/Parasagittal_MRI_of_human_head_in_patient_with_benign_familial_macrocephaly_prior_to_brain_injury_%28ANIMATED%29.gif"/>

This gif is a series of 2D images or **slices** moving from left to right. We'll now look into how we can pull slices from our 3D image

***
Sourced from: https://en.wikipedia.org/wiki/Neuroimaging#/media/File:Parasagittal_MRI_of_human_head_in_patient_with_benign_familial_macrocephaly_prior_to_brain_injury_(ANIMATED).gif

***

Note that we have two ways of working with neuroimaging data. 


1. We have the actual <code>t1_img</code> we loaded using <code>nibabel</code>
2. We have the array we pulled from <code>t1_img</code> in the form of <code>t1_data</code>


For each of these representations of our data we can pull "slices" using different methods.


First we'll work with the array in the form of <code>t1_img</code>. Since this is a <code>numpy</code> array, we can use the same method as we'd use when dealing with 3D arrays in python (R and MATLAB are very similar here!).

In [ ]:
#Pull the 10th "sagittal slice"
sagittal_slice = t1_data[10,:,:]

Notice the following structure: 

1. We can **index** an array using the array followed by square brackets as follows t1_data[x,y,z]

With our data:
- <code>x</code> is the left to right index
- <code>y</code> is the back to front index
- <code>z</code> is the bottom to top index


So here we're selecting the 10th slice going from left to right (there are 176 slices going from left to right!)

The <code>:</code>, indicates that we want to grab *everything*. 

In plain english we want to:

**Move 10 spots from the left, then grab a full 2D picture here**

**Exercise**

Now try selecting the 70th slice from the back (this is called a **coronal slice**).
It helps to think of this as follows:

**Move 70 spots from the back, then grab the full 2D picture here**. 

In [ ]:
coronal_slice = t1_data[:,70,:]

Finally try grabbing a **axial slice**, specifically the 126th slice from the bottom:

In [ ]:
axial_slice = t1_data[:,:,126]

As mentioned earlier we can also slice using <code>t1_img</code>, which we loaded through <code>nibabel</code>. It's sometimes nicer to work with this since we don't have to deal with arrays directly like we did with <code>t1_data</code>. 

Slicing with <code>t1_img</code> is just as straightforward and involves using <code>t1_img.slicer[x,y,z]</code>, but with a single caveat which we'll demonstrate here using the 10th slice from the left:

In [ ]:
sagittal_slice = t1_img.slicer[10:11,:,:]

Notice the difference? Here we have to format a sagittal slice as follows:

<code>t1_img.slicer[ (slice) :  (slice# +1) ,:,:]</code>

This is just a result of how <code>slicer</code> is programmed, but everything else remains the same. 

**Exercise**

Use <code>t1_img.slicer</code> to select the:

1. 70th coronal slice from the back
2. 126th axial slice from the bottom

In [ ]:
#Coronal slice
coronal_slice = t1_img.slicer[:,70:71,:]

In [ ]:
#Axial Slice
axial_slice = t1_img.slicer[:,:,126:127]

We've been slicing and dicing brain images but we have no idea what they look like! In the next section we'll show you how you can visualize brain slices using <code>nibabel</code>!

#### Visualizing
If we wanted to get a quick view of our data, Nibabel provides this functionality through a method called <code>orthoview()</code>. Here's how it looks in action

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
t1_img.orthoview()

But what if we wanted to view *the central slice?*. With <code>orthoview()</code>, we can only get a quick peek at the data. Nibabel also provides a tool called <code>nib.viewers.OrthoSlicer3D</code> which can help us be a bit more picky about how we want to view our images:

In [ ]:
slicer_viewer = nib.viewers.OrthoSlicer3D(t1_img.get_data())
slicer_viewer.set_position(x=86,y=150,z=160)

#### Reshaping

NumPy has a `reshape()` function for reshaping the data array. Let's say we want to convert this 3D array into a a 2D array.

In [ ]:
t1_data_2d = t1_data.reshape(np.prod(t1_data.shape[:-1]), t1_data.shape[-1])
t1_data_2d.shape

### Masks
Next, we will see how to segment the brain from the black background.

In [ ]:
plt.hist(t1_data.flatten(), bins = 50)

In [ ]:
t1_mask = t1_data > 100

In [ ]:
plt.imshow(t1_mask[87, :, :], cmap = 'gray')

In [ ]:
test = np.where(t1_mask, t1_data, 0)
plt.imshow(test[87, :, :], cmap = 'gray')

### Writing NIfTI Images

Let's save the mask we just created to a file.

In [ ]:
img_mask = nib.Nifti1Image(test, t1_affine, t1_hdr)

In [ ]:
img_mask.to_filename('../data/test_mask.nii.gz')